In [1]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import numpy as np
import itertools
import time
import random
import re

from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.datasets import make_blobs

import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize
from nltk.collocations import *

import warnings
warnings.filterwarnings('ignore', message='Polyfit*')

In [10]:
"""Read in data.

"""

df = pd.read_csv('autos.csv', sep=',', header=0, encoding='cp1252')
print(df.shape)
df.head(8)

(371528, 20)


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,privat,Angebot,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,Peugeot_206_CC_110_Platinum,privat,Angebot,2200,test,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,VW_Derby_Bj_80__Scheunenfund,privat,Angebot,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58


In [11]:
"""Clean data

i. drop data.

"""

df.drop(['dateCrawled', 'name', 'seller', 'offerType', 'abtest', 'model', 'notRepairedDamage', 'monthOfRegistration', 'dateCreated', 'nrOfPictures', 'lastSeen'], axis=1, inplace=True)
print(df.shape)

(371528, 9)


In [12]:
"""Clean data

ii. Check NaNs.drop NaNs.

"""

print(df.isnull().any()) # No NAN. If there is any, comment out the following. 
print()

print(df.shape)


price                 False
vehicleType            True
yearOfRegistration    False
gearbox                True
powerPS               False
kilometer             False
fuelType               True
brand                 False
postalCode            False
dtype: bool

(371528, 9)


In [13]:
"""Clean data

iii. Drop NaNs.

"""
# Nans in vehivle type:
index_list = df['vehicleType'].index[df['vehicleType'].isnull() == True].tolist()
# print("Index that containing special characters:\n{}".format(index_list))

df=df.drop(np.array(index_list))

# Need to reset the index, otherwise the index are gone after the drop. 
df = df.reset_index(drop=True)
print("Dataframe shape after dropping the NaNs in vehicleType:\n{}".format(df.shape))


# Nans in fuel type
index_list = df['fuelType'].index[df['fuelType'].isnull() == True].tolist()
# print("Index that containing special characters:\n{}".format(index_list))

df=df.drop(np.array(index_list))

# Need to reset the index, otherwise the index are gone after the drop. 
df = df.reset_index(drop=True)
print("Dataframe shape after dropping the NaNs in fuelType:\n{}".format(df.shape))


# Nans in gearbox
index_list = df['gearbox'].index[df['gearbox'].isnull() == True].tolist()
# print("Index that containing special characters:\n{}".format(index_list))

df=df.drop(np.array(index_list))

# Need to reset the index, otherwise the index are gone after the drop. 
df = df.reset_index(drop=True)
print("Dataframe shape after dropping the NaNs in fuelType:\n{}".format(df.shape))

Dataframe shape after dropping the NaNs in vehicleType:
(333659, 9)
Dataframe shape after dropping the NaNs in fuelType:
(317768, 9)
Dataframe shape after dropping the NaNs in fuelType:
(309900, 9)


In [14]:
"""Set the categorical data to string.


"""

num_col = ['price', 'powerPS', 'kilometer']
cat_col = ['yearOfRegistration']

"""for column in num_col:
    mean = df[column].mean()
    std = df[column].std()
    df[column] = (df[column]-mean)/std"""


df[cat_col] = df[cat_col].astype(str)

df.head(8)


,price,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,fuelType,brand,postalCode
0,18300,coupe,2011,manuell,190,125000,diesel,audi,66954
1,9800,suv,2004,automatik,163,125000,diesel,jeep,90480
2,1500,kleinwagen,2001,manuell,75,150000,benzin,volkswagen,91074
3,3600,kleinwagen,2008,manuell,69,90000,diesel,skoda,60437
4,650,limousine,1995,manuell,102,150000,benzin,bmw,33775
5,2200,cabrio,2004,manuell,109,150000,benzin,peugeot,67112
6,0,limousine,1980,manuell,50,40000,benzin,volkswagen,19348
7,14500,bus,2014,manuell,125,30000,benzin,ford,94505


In [15]:
""" Please Ignore, this is just for further reference for the final project. 

"""

""" From HW3, I know that the model is not working well, thus I decided to drop certain vehicle types. 

"""

"""lst_v = df['vehicleType'].tolist()
lst_b = df['brand'].tolist()
#v_type = list(set(lst))
#print(v_type)

from collections import Counter

count_v = Counter(lst_v)
count_b = Counter(lst_b)

print(count_v)
print()

print(count_b)"""

Counter({'limousine': 89859, 'kleinwagen': 72650, 'kombi': 62828, 'bus': 28501, 'cabrio': 21443, 'coupe': 17768, 'suv': 14002, 'andere': 2849})

Counter({'volkswagen': 64633, 'bmw': 35305, 'opel': 32107, 'mercedes_benz': 31003, 'audi': 28601, 'ford': 20981, 'renault': 14240, 'peugeot': 9370, 'fiat': 7719, 'seat': 5796, 'skoda': 5091, 'mazda': 4786, 'citroen': 4353, 'nissan': 4225, 'toyota': 4152, 'smart': 3888, 'hyundai': 3188, 'mini': 3106, 'volvo': 2946, 'mitsubishi': 2515, 'sonstige_autos': 2465, 'honda': 2298, 'kia': 2217, 'porsche': 1986, 'suzuki': 1960, 'alfa_romeo': 1957, 'chevrolet': 1595, 'chrysler': 1237, 'dacia': 803, 'jeep': 711, 'land_rover': 707, 'subaru': 648, 'daihatsu': 642, 'jaguar': 560, 'saab': 472, 'daewoo': 425, 'lancia': 384, 'rover': 365, 'trabant': 290, 'lada': 173})


In [42]:
""" Please Ignore, this is just for further reference for the final project. 

"""


""" For simplification, if a vehicle brand did not appear with a possibility of 1/800, drop it. 



dict_b = dict(count_b)
print(dict_b)

arr = np.array(list(count_b.values()))
total = np.sum(arr)

lst_drop = []
for key, value in dict_b.items():
    if value <= (total/800):
        lst_drop.append(key)

print(lst_drop)"""

{'audi': 28500, 'jeep': 709, 'volkswagen': 63960, 'skoda': 5065, 'bmw': 35085, 'peugeot': 9292, 'ford': 20778, 'mazda': 4735, 'nissan': 4188, 'renault': 14032, 'mercedes_benz': 30741, 'seat': 5733, 'honda': 2274, 'fiat': 7555, 'mini': 3090, 'smart': 3885, 'opel': 31887, 'alfa_romeo': 1944, 'subaru': 645, 'volvo': 2940, 'mitsubishi': 2492, 'hyundai': 3178, 'kia': 2194, 'suzuki': 1948, 'lancia': 384, 'porsche': 1975, 'citroen': 4240, 'toyota': 4132, 'chevrolet': 1550, 'dacia': 797, 'sonstige_autos': 2294, 'daihatsu': 630, 'saab': 471, 'chrysler': 1226, 'jaguar': 560, 'rover': 361, 'daewoo': 421, 'land_rover': 703, 'trabant': 286, 'lada': 171}
['rover', 'trabant', 'lada']


In [16]:
""" Convert to USD:

"""
df['usd'] = df['price'] * 1.23
df.head(8)

,price,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,fuelType,brand,postalCode,usd
0,18300,coupe,2011,manuell,190,125000,diesel,audi,66954,22509.0
1,9800,suv,2004,automatik,163,125000,diesel,jeep,90480,12054.0
2,1500,kleinwagen,2001,manuell,75,150000,benzin,volkswagen,91074,1845.0
3,3600,kleinwagen,2008,manuell,69,90000,diesel,skoda,60437,4428.0
4,650,limousine,1995,manuell,102,150000,benzin,bmw,33775,799.5
5,2200,cabrio,2004,manuell,109,150000,benzin,peugeot,67112,2706.0
6,0,limousine,1980,manuell,50,40000,benzin,volkswagen,19348,0.0
7,14500,bus,2014,manuell,125,30000,benzin,ford,94505,17835.0


In [17]:
# Create the price ranges with 10% adding/subtracting from the original prices. 
df['min'] = df['usd']*0.9
df['max'] = df['usd']*1.1

# Round up to the nearest 100:
df['standard_min'] = (df['min'] / 100).astype(int) *100
df['standard_max'] = (df['max'] / 100).astype(int) *100

df.head(8)

,price,vehicleType,yearOfRegistration,gearbox,powerPS,kilometer,fuelType,brand,postalCode,usd,min,max,standard_min,standard_max
0,18300,coupe,2011,manuell,190,125000,diesel,audi,66954,22509.0,20258.10,24759.90,20200,24700
1,9800,suv,2004,automatik,163,125000,diesel,jeep,90480,12054.0,10848.60,13259.40,10800,13200
2,1500,kleinwagen,2001,manuell,75,150000,benzin,volkswagen,91074,1845.0,1660.50,2029.50,1600,2000
3,3600,kleinwagen,2008,manuell,69,90000,diesel,skoda,60437,4428.0,3985.20,4870.80,3900,4800
4,650,limousine,1995,manuell,102,150000,benzin,bmw,33775,799.5,719.55,879.45,700,800
5,2200,cabrio,2004,manuell,109,150000,benzin,peugeot,67112,2706.0,2435.40,2976.60,2400,2900
6,0,limousine,1980,manuell,50,40000,benzin,volkswagen,19348,0.0,0.00,0.00,0,0
7,14500,bus,2014,manuell,125,30000,benzin,ford,94505,17835.0,16051.50,19618.50,16000,19600


In [ ]:
""" Convert type to English:

{'kleinwagen': Hatchback, 'cabrio': convertible, 'coupe': sedan, 'suv': suv}

"""


In [18]:
# Drop unneccessary:
df.drop(['gearbox', 'powerPS', 'fuelType', 'postalCode', 'price', 'usd', 'min', 'max'], axis=1, inplace=True)
df.head(8)

,vehicleType,yearOfRegistration,kilometer,brand,standard_min,standard_max
0,coupe,2011,125000,audi,20200,24700
1,suv,2004,125000,jeep,10800,13200
2,kleinwagen,2001,150000,volkswagen,1600,2000
3,kleinwagen,2008,90000,skoda,3900,4800
4,limousine,1995,150000,bmw,700,800
5,cabrio,2004,150000,peugeot,2400,2900
6,limousine,1980,40000,volkswagen,0,0
7,bus,2014,30000,ford,16000,19600


In [19]:
df.to_csv('Cleaned_data.csv', encoding='utf-8') # Remember to delete the index column. 

"""Noted that I used 'data filter' inside the Cleaned_data.csv file to keep only the desired brands and types.

"""

In [20]:
""" Read in the newly created dataset for cleaned data:


"""
data = pd.read_csv('HIT.csv', sep=',', header=0)
print(data.shape)
data.head(8)

(8803, 7)


,ID,vehicleType,yearOfRegistration,kilometer,brand,standard_min,standard_max
0,11,suv,2011,70000,nissan,19900,24300
1,38,Sedan,2014,30000,bmw,43800,53500
2,53,Hatchback,2013,40000,nissan,8800,10800
3,75,Sedan,2012,50000,hyundai,25400,31100
4,89,Hatchback,2010,40000,volkswagen,10200,12500
5,109,suv,2014,40000,mercedes_benz,46300,56600
6,197,Hatchback,2012,30000,volkswagen,13100,16100
7,213,Sedan,2014,80000,mercedes_benz,53100,64900


In [21]:
""" Randomly choose 20 cars of each selected brand:

brands = ['volkswagen', 'bmw', 'mercedes_benz', 'ford', 'mazda', 'nissan', 'toyota', 'hyundai', 'honda', 'chevrolet']


"""

brands = ['volkswagen', 'bmw', 'mercedes_benz', 'ford', 
          'mazda', 'nissan', 'toyota', 'hyundai', 'honda', 'chevrolet']

brand_lst = []
for brand in brands:
    temp = (data[data['brand']==brand]).sample(n=20)
    brand_lst.append(temp)

sample = pd.concat(brand_lst)
#print(sample.shape)

from sklearn.utils import shuffle

np.random.seed(0)
sample = shuffle(sample)
print(sample.shape)
sample = sample.reset_index(drop=True)
sample.head(8)


(200, 7)


,ID,vehicleType,yearOfRegistration,kilometer,brand,standard_min,standard_max
0,159847,Hatchback,2012,80000,volkswagen,8800,10800
1,106475,Sedan,2012,70000,honda,11000,13500
2,306655,Hatchback,2012,60000,nissan,6400,7800
3,295298,suv,2010,150000,mazda,13200,16200
4,57304,Hatchback,2012,50000,honda,10200,12500
5,50992,Hatchback,2013,30000,chevrolet,7500,9200
6,67870,Hatchback,2010,125000,volkswagen,7000,8500
7,200171,Hatchback,2013,30000,hyundai,9900,12100


In [23]:
sample.to_csv('Samples.csv', encoding='utf-8') # Remember to delete the index column. 

In [77]:
""" Now that I have created GOLD_50.csv to hold my 50 gold hits, I need to insert them randomly.

First, read in GOLD_50.csv and Samples.csv

Then, concatenate the dataframes in order to insert the gold hits. 

"""

gold = pd.read_csv('GOLD_50.csv', sep=',', header=0)
samples = pd.read_csv('Samples.csv', sep=',', header=0) # Need to read in again, since I added index in thi csv.

# Insert the gold hits with concatnation, therefore, further shuffling is needed. 
combine = pd.concat([gold, samples])

from sklearn.utils import shuffle

np.random.seed(0)
gold = shuffle(gold) # Since the gold hits are created in order, I need to shuffle it first.
samples = shuffle(samples)

combine = shuffle(combine) # Shuffle the inserted data. 
print(combine.shape)
combine = combine.reset_index(drop=True)
combine.head(25)

(250, 7)


,ID,brand,kilometer,standard_max,standard_min,vehicleType,yearOfRegistration
0,192198,toyota,20000,200,100,Hatchback,2014
1,260313,ford,70000,7000,5700,Hatchback,2012
2,14378,toyota,10000,11500,9400,Hatchback,2014
3,297711,volkswagen,60000,29600,24200,suv,2012
4,159525,chevrolet,30000,6400,5300,Hatchback,2010
5,100952,hyundai,30000,9400,7700,Hatchback,1900
6,98548,mercedes_benz,70000,24200,19800,Sedan,2011
7,35082,mazda,10000,14200,11600,Hatchback,1900
8,99308,mercedes_benz,60000,60800,49800,suv,2014
9,266934,volkswagen,150000,16200,13200,suv,2011


In [78]:
combine.to_csv('HIT_Final.csv', encoding='utf-8') # Remember to delete the index column.

In [79]:
""" Now I need to combine multiple rows into one row for the purpose of creation of one HIT. 

"""

final = pd.read_csv('HIT_Final.csv', sep=',', header=0)
final = shuffle(final)

In [83]:
""" To create the content in one cell, I created a dictionary to hold all the information.

"""
lst = []
for i in range(len(final['ID'])):
    temp = {'Brand': final['brand'].iloc[i],
            'Type': final['vehicleType'].iloc[i],
            'Year': final['yearOfRegistration'].iloc[i],
            'Mileage': final['kilometer'].iloc[i],
            'Price Range': '${} ~ ${}'.format(final['standard_min'].iloc[i], final['standard_max'].iloc[i])
           }
    lst.append(temp) # Create a list to hold all the dictionary for further uses. 
    
#print(lst)

""" Split the list created above into 10 batches.

"""

size = 10
seq = lst
test = [seq[i:i+size] for i  in range(0, len(seq), size)]

#print(test)

# Creating column names: 1 to 25.
columns = list(range(1,26))


# Create data dictionary for further creation of dataframe.
data = {}
for col,d in zip(columns, test):
    data[col] = d
    
#print(data)
    

{1: [{'Brand': 'ford', 'Type': 'Hatchback', 'Year': 2012, 'Mileage': 70000, 'Price Range': '$5700 ~ $7000'}, {'Brand': 'ford', 'Type': 'suv', 'Year': 2014, 'Mileage': 90000, 'Price Range': '$22100 ~ $27000'}, {'Brand': 'mercedes_benz', 'Type': 'Sedan', 'Year': 2011, 'Mileage': 5000, 'Price Range': '$33000 ~ $40300'}, {'Brand': 'volkswagen', 'Type': 'Hatchback', 'Year': 2011, 'Mileage': 70000, 'Price Range': '$8000 ~ $9800'}, {'Brand': 'hyundai', 'Type': 'suv', 'Year': 2011, 'Mileage': 125000, 'Price Range': '$15300 ~ $18700'}, {'Brand': 'nissan', 'Type': 'Hatchback', 'Year': 2011, 'Mileage': 80000, 'Price Range': '$9400 ~ $11500'}, {'Brand': 'volkswagen', 'Type': 'convertible', 'Year': 1900, 'Mileage': 20000, 'Price Range': '$26000 ~ $31700'}, {'Brand': 'volkswagen', 'Type': 'convertible', 'Year': 2012, 'Mileage': 30000, 'Price Range': '$18500 ~ $22700'}, {'Brand': 'toyota', 'Type': 'suv', 'Year': 2010, 'Mileage': 150000, 'Price Range': '$31500 ~ $38500'}, {'Brand': 'mercedes_benz', 'T

In [96]:
""" Write dataframe from the data dictionary created above and write the created dataframe into a csv file for the MTurk.

"""
df = DataFrame(data)

df.to_csv("HW4_HIT.csv", encoding='utf-8')

In [ ]:
"""
*************************************************Results Exploration********************************************************
"""

In [119]:
# I have identified which car is the gold hit for each assignment, and put a list of car numbers in 'test' column. 
res = pd.read_csv('results.csv', sep=',', header=0)


<class 'str'>


In [120]:
""" Determine the gold hit accuracy.

Step 1:
Read in the 'test' column, and check the answers in the columns that were mentioned in 'test', which are the gold hits. 

Step 2: Determine the gold hit accuracy:

"""

import json


for ele in range(len(res['test'])):
    print("**************** GOLD HIT Results in HIT #{} *****************".format((ele+1)))
    
    # Read in as json, since the list is stored as str.
    test = json.loads(res['test'][ele])
    for t in test:
        answer = res['Answer.car{}'.format(t)][ele]
        print("Answer for car {}: {}".format(t, answer))


**************** GOLD HIT Results in HIT #1 *****************
Answer for car 6: Information Not Correct
Answer for car 13: Information Not Correct
Answer for car 14: Information Not Correct
Answer for car 18: Information Not Correct
**************** GOLD HIT Results in HIT #2 *****************
Answer for car 6: Too High
Answer for car 13: Reasonable
Answer for car 14: Information Not Correct
Answer for car 18: Information Not Correct
**************** GOLD HIT Results in HIT #3 *****************
Answer for car 6: Too Low
Answer for car 13: Too Low
Answer for car 14: Too Low
Answer for car 18: Reasonable
**************** GOLD HIT Results in HIT #4 *****************
Answer for car 15: Information Not Correct
Answer for car 17: Information Not Correct
Answer for car 18: Reasonable
**************** GOLD HIT Results in HIT #5 *****************
Answer for car 15: Information Not Correct
Answer for car 17: Information Not Correct
Answer for car 18: Information Not Correct
**************** GOLD

In [128]:
""" Determine the gold hit accuracy.

Step 2: Determine the gold hit accuracy:

"""

for ele in range(len(res['test'])):
    test = json.loads(res['test'][ele])
    corr =  0
    for t in test:
        answer = res['Answer.car{}'.format(t)][ele]
        if answer == 'Information Not Correct':
            corr += 1
            
    accuracy = (corr/(len(test)))*100
    print("Accuracy for HIT {}: {}%".format(ele+1, np.round(accuracy, 2)))


Accuracy for HIT 1: 100.0%
Accuracy for HIT 2: 50.0%
Accuracy for HIT 3: 0.0%
Accuracy for HIT 4: 66.67%
Accuracy for HIT 5: 100.0%
Accuracy for HIT 6: 66.67%
Accuracy for HIT 7: 100.0%
Accuracy for HIT 8: 0.0%
Accuracy for HIT 9: 100.0%
Accuracy for HIT 10: 100.0%
Accuracy for HIT 11: 100.0%
Accuracy for HIT 12: 100.0%
Accuracy for HIT 13: 100.0%
Accuracy for HIT 14: 100.0%
Accuracy for HIT 15: 55.56%
Accuracy for HIT 16: 100.0%
Accuracy for HIT 17: 100.0%
Accuracy for HIT 18: 100.0%
Accuracy for HIT 19: 100.0%
Accuracy for HIT 20: 42.86%
Accuracy for HIT 21: 100.0%
Accuracy for HIT 22: 50.0%
Accuracy for HIT 23: 75.0%
Accuracy for HIT 24: 0.0%
Accuracy for HIT 25: 0.0%
Accuracy for HIT 26: 100.0%
Accuracy for HIT 27: 100.0%
Accuracy for HIT 28: 100.0%
Accuracy for HIT 29: 100.0%
Accuracy for HIT 30: 75.0%


In [134]:
""" Time spent on each HIT:

Convert seconds to min + second.

"""
for ele in range(len(res['time'])):
    
    # Convert to minute + seconds:
    minute = int((res['time'][ele])/60)
    second = res['time'][ele] - minute*60
    print("Time for HIT {}: {} minut {} seconds".format(ele+1, minute, second))


Time for HIT 1: 8 minut 14 seconds
Time for HIT 2: 5 minut 49 seconds
Time for HIT 3: 5 minut 59 seconds
Time for HIT 4: 14 minut 24 seconds
Time for HIT 5: 14 minut 39 seconds
Time for HIT 6: 11 minut 42 seconds
Time for HIT 7: 9 minut 30 seconds
Time for HIT 8: 12 minut 11 seconds
Time for HIT 9: 11 minut 1 seconds
Time for HIT 10: 9 minut 12 seconds
Time for HIT 11: 9 minut 6 seconds
Time for HIT 12: 7 minut 27 seconds
Time for HIT 13: 8 minut 35 seconds
Time for HIT 14: 4 minut 24 seconds
Time for HIT 15: 6 minut 47 seconds
Time for HIT 16: 11 minut 38 seconds
Time for HIT 17: 8 minut 58 seconds
Time for HIT 18: 7 minut 22 seconds
Time for HIT 19: 9 minut 37 seconds
Time for HIT 20: 7 minut 30 seconds
Time for HIT 21: 6 minut 49 seconds
Time for HIT 22: 7 minut 50 seconds
Time for HIT 23: 12 minut 40 seconds
Time for HIT 24: 5 minut 26 seconds
Time for HIT 25: 9 minut 53 seconds
Time for HIT 26: 5 minut 50 seconds
Time for HIT 27: 11 minut 15 seconds
Time for HIT 28: 12 minut 28 se

In [146]:
""" Descriptive statistics on the results. 

Check the answer frequency for each HIT. 

"""

answers = []
lows = []
highs = []
rights = []
for ele in range(len(res['test'])):
    #print("**************** Results in HIT #{} *****************".format((ele+1)))
    test = json.loads(res['test'][ele])
    low = 0
    right = 0
    high = 0    
    
    # Check frequecy for each car records.
    for t in range(1,26):
        if t not in test:
            answer = res['Answer.car{}'.format(t)][ele]
            if answer == 'Too Low':
                low += 1
            elif answer == 'Reasonable':
                right += 1
            elif answer == 'Too High':
                high += 1
            
    #print("Answer for car {}: {}".format(t, answer))
    ans = [low, right, high]
    answers.append(ans)
    lows.append(low)
    rights.append(right)
    highs.append(high)
    #print(ans)

# print(answers)
data = {"Too Low": lows,
        "Reasonable": rights,
        "Too high": highs}

df = pd.DataFrame(data)
df.head(30)


,Reasonable,Too Low,Too high
0,7,0,14
1,13,1,7
2,13,6,2
3,16,1,5
4,13,2,7
5,9,3,10
6,13,2,8
7,7,2,14
8,12,1,10
9,4,1,13


In [269]:
""" For Inter Rater: Count each class for each car record.

Noted that this task needs special treatment, since each assignment contains up to 25 car records;
And from the first assignment, every 3 assignments have the same batch of car records (which allows 3 workers to work on one car.)

According to the code below, the GOLD HITs contained here should be coded as [0, 0, 0] for all three categories.

The results from the code are double-checked with the results csv files, and I am sure it is working. 

"""

data_lows = []
data_highs = []
data_rights = []

for ele in range(len(res['test'])):
    test = json.loads(res['test'][ele])
    lows = []
    highs = []
    rights = []
    for t in range(1,26):
        low = 0
        right = 0
        high = 0
        if t not in test:
            answer = res['Answer.car{}'.format(t)][ele]
            if answer == 'Too Low':
                low += 1
            elif answer == 'Reasonable':
                right += 1
            elif answer == 'Too High':
                high += 1
        lows.append(low)
        rights.append(right)
        highs.append(high)
    #print(lows, rights, highs)
    data_lows.append(lows)
    data_rights.append(rights)
    data_highs.append(highs)

datas = [data_lows, data_rights, data_highs]
low_a, right_a, high_a = [], [], []
ann = [low_a, right_a, high_a]

for d, a in zip(datas, ann):
    arr = np.array(d)
    for ele in range(3, 31, 3):
        ele_sum = []
        
        # For every 3 rows, they are from the same HIT.
        for n in list(range(25)):
            s = arr[(ele-3):(ele),n].sum() # Count each estimation in the same HIT (3 rows)
            ele_sum.append(s)
        a.extend(ele_sum)

#print(high_a)
data = {'Too Low': low_a,
        'Reasonable': right_a,
        'Too High': high_a}

IR = pd.DataFrame(data)
#print(IR)

# Drop GOLD HITs with [0,0,0]:
IR = IR[(IR.T != 0).any()]
print(IR)

     Reasonable  Too High  Too Low
0             3         0        0
1             3         0        0
2             2         1        0
3             2         1        0
4             2         1        0
6             2         0        1
7             2         0        1
8             1         2        0
9             2         1        0
10            2         0        1
11            1         2        0
14            2         1        0
15            1         1        1
16            2         0        1
18            2         1        0
19            1         2        0
20            0         3        0
21            1         2        0
22            0         2        1
23            1         2        0
24            1         1        1
25            0         3        0
26            2         0        1
27            3         0        0
28            2         0        1
29            0         2        1
30            3         0        0
31            3     

In [270]:
""" Write out data. 

"""

IR.to_csv('Inter_Rater_Final.csv', encoding='utf-8')

"""
And from now on, the Inter-rater calculation will be performed from the CSV. 

"""

'\nAnd from now on, the Inter-rater calculation will be performed from the CSV. \n\n'

In [ ]:
""" 
****************************************** Classification and Validation *****************************************************
"""

In [12]:
""" Find the reasonable price ranges from MTurk:

Need to compile the car information. I did it in Excel. So I need to read in.

"""
df = pd.read_csv("Classification_Final.csv")
df.head(8)

,Reasonable,Too High,Too Low,Car Info
0,3,0,0,"{'Brand': 'ford', 'Type': 'Hatchback', 'Year':..."
1,3,0,0,"{'Brand': 'ford', 'Type': 'Hatchback', 'Year':..."
2,2,1,0,"{'Brand': 'mazda', 'Type': 'Hatchback', 'Year'..."
3,2,1,0,"{'Brand': 'bmw', 'Type': 'Sedan', 'Year': 2013..."
4,2,1,0,"{'Brand': 'hyundai', 'Type': 'Hatchback', 'Yea..."
5,2,0,1,"{'Brand': 'toyota', 'Type': 'Hatchback', 'Year..."
6,2,0,1,"{'Brand': 'chevrolet', 'Type': 'Hatchback', 'Y..."
7,1,2,0,"{'Brand': 'hyundai', 'Type': 'suv', 'Year': 20..."


In [13]:
""" Find reasonable price ranges by 'simple majority' principle.

"""


# Sort out the 'resonable' car info:
df = df['Car Info'][df['Reasonable']>1]

# Need to reset the index:
df = df.reset_index(drop=True)
df.head(8)

0    {'Brand': 'ford', 'Type': 'Hatchback', 'Year':...
1    {'Brand': 'ford', 'Type': 'Hatchback', 'Year':...
2    {'Brand': 'mazda', 'Type': 'Hatchback', 'Year'...
3    {'Brand': 'bmw', 'Type': 'Sedan', 'Year': 2013...
4    {'Brand': 'hyundai', 'Type': 'Hatchback', 'Yea...
5    {'Brand': 'toyota', 'Type': 'Hatchback', 'Year...
6    {'Brand': 'chevrolet', 'Type': 'Hatchback', 'Y...
7    {'Brand': 'bmw', 'Type': 'convertible', 'Year'...
Name: Car Info, dtype: object

In [14]:
""" Create training test:

Only 79 left.

"""

brands = []
types = []
years = []
miles = []
ranges = []

for ele in range(len(df)):
    # Convert str into dictionary:
    test = eval(df[ele])
    brands.append(test['Brand'])
    types.append(test['Type'])
    years.append(test['Year'])
    miles.append(test['Mileage'])
    ranges.append(test['Price Range'])

data = {'Brand': brands,
        'Type': types,
        'Year': years,
        'Mileage': miles,
        'Price Ranges': ranges}

df = pd.DataFrame(data)
#print(len(df['Brand']))
df.head(8)
    

,Brand,Mileage,Price Ranges,Type,Year
0,ford,70000,$5700 ~ $7000,Hatchback,2012
1,ford,10000,$7700 ~ $9400,Hatchback,2012
2,mazda,50000,$9900 ~ $12100,Hatchback,2012
3,bmw,30000,$21000 ~ $25700,Sedan,2013
4,hyundai,70000,$6300 ~ $7700,Hatchback,2010
5,toyota,30000,$5200 ~ $6300,Hatchback,2014
6,chevrolet,30000,$7500 ~ $9200,Hatchback,2013
7,bmw,60000,$32700 ~ $40000,convertible,2012


In [15]:
""" Convert strings in column 'Price Ranges' into price integers.

And calculate the median for the price point directly. 

"""
price_point = []
for i in range(len(df['Price Ranges'])):
    s = df['Price Ranges'][i]
    parts = s.split("~")
    prices = []
    for t in parts:
        temp = ''.join([x for x in t if x.isdigit()])
        prices.append(int(temp))
    #print(prices)
    
    # Creating median as the price point.
    price = np.median(prices)
    price_point.extend([price])
    #print(price)
    
#print(len(price_point))
df['price'] = price_point


In [16]:
df.head(8)

,Brand,Mileage,Price Ranges,Type,Year,price
0,ford,70000,$5700 ~ $7000,Hatchback,2012,6350.0
1,ford,10000,$7700 ~ $9400,Hatchback,2012,8550.0
2,mazda,50000,$9900 ~ $12100,Hatchback,2012,11000.0
3,bmw,30000,$21000 ~ $25700,Sedan,2013,23350.0
4,hyundai,70000,$6300 ~ $7700,Hatchback,2010,7000.0
5,toyota,30000,$5200 ~ $6300,Hatchback,2014,5750.0
6,chevrolet,30000,$7500 ~ $9200,Hatchback,2013,8350.0
7,bmw,60000,$32700 ~ $40000,convertible,2012,36350.0


In [23]:
""" Classification:

Use linear regression first, and accuracy scores as the metrics.

Categorical columns: Brand, Type, and Year
Numerical columns: Mileage, Price.

"""
data_col = ['Brand', 'Type', 'Year', 'Mileage']
Original_X = df[data_col]
y=df['price']

# Since there are categorical variables, we need to Dict_Vectorize the X: 
dict_data = pd.DataFrame(Original_X).T.to_dict().values()

# Initiate vectorizer
vectorizer = DictVectorizer(sparse=False)

# Transform X and y: y does not need to transform. 
X = pd.DataFrame(vectorizer.fit_transform(dict_data))

# Testing and Training split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [24]:
""" 10-fold cross validation: Using all data.

"""

classifier = linear_model.LinearRegression()
# Fit the model with all samples, in preparation for the cross validation and AIC BIC.
classifier.fit(X, y)

# Print the accuracy scores:
print("Acuracy scores: {}".format(np.round(classifier.score(X,y), 2)))
print()
    
scores = cross_val_score(classifier, X, y, cv=10)
print("10 fold scores are")
print(np.round(scores, 2))
print()
print("Average Accuracy = %.2f" % np.mean(scores))
print()


Acuracy scores: 0.81

10 fold scores are
[ 0.23  0.12 -0.05  0.85  0.63  0.79  0.77  0.55  0.54  0.96]

Average Accuracy = 0.54



In [25]:
""" R2 and MSE: Use testing and trainig dataste. 

"""
classifier.fit(X_train, y_train)

# Prediction
y_pred = classifier.predict(X_test)

print('Variance score: %.2f' % r2_score(y_test, y_pred))
print()
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Variance score: 0.89

Mean squared error: 18329060.52


In [30]:
""" Classification:

"""

svr = svm.SVC()
svr.fit(X_train, y_train)

y_pred = svr.predict(X_test)
# Print the metrics report.
print("Classification report for classifier %s\n%s\n" % (svr, metrics.classification_report(y_test, y_pred)))


Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

     5350.0       0.00      0.00      0.00         1
     5750.0       0.00      0.00      0.00         0
     7000.0       0.50      1.00      0.67         1
     8300.0       0.00      0.00      0.00         1
     8350.0       0.00      0.00      0.00         1
     8500.0       0.00      0.00      0.00         0
     8550.0       0.00      0.00      0.00         0
     8800.0       0.00      0.00      0.00         0
     9550.0       0.00      0.00      0.00         1
    11000.0       0.00      0.00      0.00         1
    11350.0       0.00      0.00      0.00         1
    11550.0       0.00      0.00      0.00         1
    12900.0       0.00      0.00      0.00      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
